In [40]:
import pymssql
import pandas as pd
import numpy as np
import requests
import json
from datetime import datetime
from functions import *
import sqlalchemy
from sqlalchemy  import create_engine
from geopy import Point
from geopy.distance import distance
import random

# Local DB PC
server = 'DESKTOP-3OHRULK'
user = 'sa'
password = 'welcome1'
db = 'pfidb'


conn = pymssql.connect(server,user,password,db)

conn_for_insert = fr'mssql+pymssql://'+user+':'+password+'@'+server+'/'+db
engine = create_engine(conn_for_insert)

In [2]:
stations_query = """SELECT * FROM [estaciones-de-bicicletas-publicas]"""
df_stations = pd.read_sql(stations_query,conn)

df_stations_usage = stations_usage(conn,df_stations)
stations_avg_df = df_stations_usage.groupby('nro_est')['bicicletas_en_estacion','usos'].mean()
stations_avg_df = stations_avg_df.sort_values('bicicletas_en_estacion').reset_index()
stations_avg_df.head(10)

,nro_est,bicicletas_en_estacion,usos
0,33.0,-527.638239,4.995637
1,5.0,-514.242713,3.467381
2,131.0,-379.331877,3.264010
3,89.0,-352.976258,2.316129
4,9.0,-325.316999,3.998101
5,57.0,-272.925195,1.990958
6,22.0,-244.444320,1.899477
7,199.0,-212.869172,2.582617
8,200.0,-210.182254,2.574133
9,28.0,-207.420819,2.865709


In [3]:
stations_avg_month_df = df_stations_usage.copy()
stations_avg_month_df['year_month'] = stations_avg_month_df['fecha_y_hora'].apply(lambda x: x.strftime('%Y-%m'))
stations_avg_month_df = stations_avg_month_df.groupby(['nro_est','year_month'])['bicicletas_en_estacion','usos'].mean()
stations_avg_month_df = stations_avg_month_df.sort_values('bicicletas_en_estacion').reset_index()
stations_avg_month_df.head(10)

,nro_est,year_month,bicicletas_en_estacion,usos
0,5.0,2018-12,-1050.555263,2.363158
1,33.0,2018-12,-958.167665,3.211577
2,5.0,2018-11,-950.578189,3.742798
3,33.0,2018-11,-889.177007,5.348540
4,33.0,2018-10,-779.087687,7.604478
5,5.0,2018-10,-776.240575,4.524237
6,131.0,2018-12,-713.849727,2.743169
7,9.0,2018-12,-713.272517,2.965358
8,33.0,2018-09,-692.228663,8.185185
9,131.0,2018-11,-653.013015,3.865510


In [4]:
K = defining_right_k(df_stations)
print(K)

24


In [5]:
df_merged = cluster(df_stations, K) # Cluster into K groups

In [6]:
# Calculating and adding the radius to each cluster
df_merged_radius = determing_cluster_radius(df_merged)
# df_merged_radius.rename(columns={'euclidean_distance':'radius'},inplace=True)
df_merged_radius.sort_values('cluster').head(10)

,long,lat,nombre,domicilio,nro_est,dire_norm,capacidad,cluster,lat_centroide,long_centroide,radius
99,-58.442423,-34.589049,Fitz Roy,Fitz Roy 1092 y Loyola,101,1092 FITZ ROY,12,0,-34.589985,-58.44849,0.660764
156,-58.445099,-34.592839,Villarroel,Villarroel y Humboldt,158,VILLARROEL y HUMBOLDT,12,0,-34.589985,-58.44849,0.660764
125,-58.452927,-34.592212,Santos Dumont y Otero,Santos Dumont 4380 y Otero,127,4380 DUMONT SANTOS,10,0,-34.589985,-58.44849,0.660764
102,-58.455062,-34.587548,Federico Lacroze,Av Corrientes 6800 y Federico Lacroze,104,6900 CORRIENTES AV,20,0,-34.589985,-58.44849,0.660764
100,-58.445987,-34.586440,Roseti,Roseti 50 y Av Dorrego,102,50 ROSETI,12,0,-34.589985,-58.44849,0.660764
92,-58.449442,-34.591819,Guzmin,Guzmin y Leiva,94,GUZMaN y LEIVA,20,0,-34.589985,-58.44849,0.660764
187,-58.385586,-34.588475,Posadas,Posadas 1350 entre Montevideo y Rodriguez Pena,189,1350 POSADAS,12,1,-34.588454,-58.39319,0.697636
164,-58.393317,-34.589150,Cementerio de la Recoleta,Vicente Lopez 2050 entre Junin y Uriburu,166,2050 LOPEZ VICENTE,12,1,-34.588454,-58.39319,0.697636
117,-58.397684,-34.588372,Emilio Mitre,Plaza Emilio Mitre - Larrea y Pacheco de Melo,119,LARREA y PACHECO DE MELO JOSE,12,1,-34.588454,-58.39319,0.697636
113,-58.390786,-34.587812,Quintana,Manuel Quintana 600,115,600 QUINTANA MANUEL,10,1,-34.588454,-58.39319,0.697636


In [7]:
df_merged_radius[df_merged_radius['nro_est']==42]
df_merged_radius.head(10)

,long,lat,nombre,domicilio,nro_est,dire_norm,capacidad,cluster,lat_centroide,long_centroide,radius
0,-58.392452,-34.583133,Facultad de Derecho,Av. Pres.Figueroa Alcorta y Juan A.Bibiloni,1,"FIGUEROA ALCORTA, PRES. AV. y BIBILONI, JUAN A.",20,1,-34.588454,-58.393190,0.697636
1,-58.374822,-34.592589,Retiro,Av. Dr.Jose Ramos Mejia y Del Libertador Av,2,"DEL LIBERTADOR AV. y RAMOS MEJIA, JOSE MARIA, ...",12,18,-34.594475,-58.376283,0.578273
2,-58.368918,-34.611242,Aduana,Av. Ing.Huergo y Av. Belgrano,3,"HUERGO, ING. AV. y BELGRANO AV.",12,14,-34.614912,-58.365744,0.670338
3,-58.368950,-34.601721,Plaza Roma,Lavalle y Bouchard,4,LAVALLE y BOUCHARD,12,5,-34.603205,-58.370342,0.701368
4,-58.420997,-34.580127,Plaza Italia,Av. Santa Fe y Av. Sarmiento,5,SARMIENTO AV. y CALZADA CIRCULAR PLAZA ITALIA,42,11,-34.574162,-58.416990,0.807916
5,-58.369709,-34.628301,Parque Lezama,Av Martin Garcia e Irala,6,"GARCIA, MARTIN AV. e IRALA",12,15,-34.627072,-58.373073,0.651503
6,-58.380990,-34.605840,Obelisco,Av. 9 de Julio y Tte. Gral Juan Domingo Peron,7,"PELLEGRINI, CARLOS y PERON, JUAN DOMINGO, TTE....",10,12,-34.602809,-58.382236,0.648369
7,-58.389253,-34.609930,Congreso,Av. Hipolito Yrigoyen y Virrey Ceballos,8,"YRIGOYEN, HIPOLITO AV. y CEVALLOS, VIRREY",24,13,-34.610873,-58.391128,0.544670
8,-58.407145,-34.583915,Parque Las Heras,Parque Las Heras,9,"DIAZ, CNEL. AV. y FRENCH",24,23,-34.582460,-58.406626,0.668911
9,-58.365746,-34.615551,Puerto Madero - UCA,Moreau de Justo 1500,10,"1500 MOREAU DE JUSTO, ALICIA AV.",12,14,-34.614912,-58.365744,0.670338


## Trying to add a new station near a full one

In [35]:
### Adding new station to the cluster table and generating df_merged_radius from the DB
def jsonify_usig(res):
    return json.loads(res.text[1:-1])

# ONLY NEEDS NAME, CAPACITY AND CLUSTER!!
new_station = {
    'long':0,
    'lat':0,
    'nombre':'Test 5 para variar cluster 15',
    'domicilio':'',
    'nro_est':0,
    'dire_norm':'',
    'capacidad':100,
    'cluster':15,
    'lat_centroide':0,
    'long_centroide':0
}

new_station['nro_est'] = engine.execute("SELECT MAX(nro_est)+1 FROM [estaciones-de-bicicletas-publicas]").fetchone()[0]

cluster_info_query = """SELECT lat_centroide, long_centroide 
                FROM [pfidb].[dbo].[stations_clusters_usage]
                WHERE cluster={}
                GROUP BY lat_centroide,long_centroide""".format(new_station['cluster'])

cluster_lat_long = engine.execute(cluster_info_query).fetchone()
#new_station['lat'] = cluster_lat_long[0]
#new_station['long'] = cluster_lat_long[1]
new_station['lat_centroide'] = cluster_lat_long[0]
new_station['long_centroide'] = cluster_lat_long[1]

lat_long_lists_query = """SELECT lat, long 
                FROM [pfidb].[dbo].[stations_clusters_usage]
                WHERE cluster={}""".format(new_station['cluster'])

lat_list = []
long_list = []
for i in engine.execute(lat_long_lists_query).fetchall():
    lat_list.append(i[0])
    long_list.append(i[1])

lat_aux = (random.choice(lat_list)+random.choice(lat_list))/2
while lat_aux in lat_list:
    lat_aux = (random.choice(lat_list)+random.choice(lat_list))/2


long_aux = (random.choice(long_list)+random.choice(long_list))/2
while long_aux in long_list:
    long_aux = (random.choice(long_list)+random.choice(long_list))/2

new_station['lat'] = lat_aux
new_station['long'] = long_aux

# Turning lat long to address
URL="https://ws.usig.buenosaires.gob.ar/geocoder/2.2/reversegeocoding"
# X is longitude and Y is latitude
params = {'x':new_station['long'], 'y':new_station['lat']}
usig_address = jsonify_usig(requests.get(URL,params=params))

new_station['domicilio'] = usig_address['puerta']
new_station['dire_norm'] = usig_address['puerta']


stations_clusters_query = """SELECT * FROM [stations_with_centroids]"""
df_merged = pd.read_sql(stations_clusters_query,conn)
df_merged = df_merged.append(new_station, ignore_index=True)
insert_stations_with_centroids(engine,'stations_with_centroids',df_merged)


aux_keys = ['long','lat','nombre','domicilio','nro_est','dire_norm','capacidad']
new_station_aux = { key: new_station[key] for key in aux_keys }
stations_query = """SELECT * FROM [estaciones-de-bicicletas-publicas]"""
df_stations = pd.read_sql(stations_query,conn)
df_stations = df_stations.append(new_station_aux, ignore_index=True)
generic_insert(engine,'estaciones-de-bicicletas-publicas',df_stations)

df_merged = pd.read_sql(stations_clusters_query,conn)

# print(new_station_aux)

{'long': -58.3644982275, 'lat': -34.614985403750005, 'nombre': 'Test para variar clusters', 'domicilio': 'GORRITI JUANA MANUELA 1500', 'nro_est': 201, 'dire_norm': 'GORRITI JUANA MANUELA 1500', 'capacidad': 100}


In [8]:
# Calculating how many stations are in each clusters
df_clusters = df_merged_radius.groupby("cluster")["nro_est"].apply(list).reset_index()
df_clusters["count"] = df_clusters["nro_est"].apply(len)
df_clusters.head(25)

,cluster,nro_est,count
0,0,"[94, 101, 102, 104, 127, 158]",6
1,1,"[1, 12, 30, 90, 115, 119, 166, 189]",8
2,2,"[16, 18, 24, 37, 46, 47, 72, 98, 112, 117, 149...",12
3,3,"[49, 67, 80, 97, 157, 178, 180, 185, 199]",9
4,4,"[17, 52, 54, 96, 110, 143, 154, 161, 194]",9
5,5,"[4, 26, 32, 34, 40, 48, 55, 60, 61, 63, 111, 1...",13
6,6,"[13, 28, 88, 120, 134, 146, 162, 163, 169, 186]",10
7,7,"[108, 126, 153, 176, 179]",5
8,8,"[25, 56, 65, 70, 113, 122, 123, 142]",8
9,9,"[21, 41, 92, 105, 139, 140, 155, 198]",8


In [9]:

stations_clusters_original = pd.merge(df_merged_radius,stations_avg_df, on='nro_est', how='left')
stations_clusters_original['usos'].fillna(0, inplace=True)
stations_clusters_original['bicicletas_en_estacion'].fillna(stations_clusters_original['capacidad'], inplace=True)
stations_clusters_usage = stations_clusters_original.copy()
stations_clusters_usage['very_used'] = np.where(stations_clusters_usage['bicicletas_en_estacion'] < 0, True, False)
#stations_clusters_usage.groupby('very_used').count()
stations_clusters_usage.head(10)

,long,lat,nombre,domicilio,nro_est,dire_norm,capacidad,cluster,lat_centroide,long_centroide,radius,bicicletas_en_estacion,usos,very_used
0,-58.392452,-34.583133,Facultad de Derecho,Av. Pres.Figueroa Alcorta y Juan A.Bibiloni,1,"FIGUEROA ALCORTA, PRES. AV. y BIBILONI, JUAN A.",20,1,-34.588454,-58.393190,0.697636,134.415700,3.538632,False
1,-58.374822,-34.592589,Retiro,Av. Dr.Jose Ramos Mejia y Del Libertador Av,2,"DEL LIBERTADOR AV. y RAMOS MEJIA, JOSE MARIA, ...",12,18,-34.594475,-58.376283,0.578273,-76.380952,2.258815,True
2,-58.368918,-34.611242,Aduana,Av. Ing.Huergo y Av. Belgrano,3,"HUERGO, ING. AV. y BELGRANO AV.",12,14,-34.614912,-58.365744,0.670338,68.653128,1.668496,False
3,-58.368950,-34.601721,Plaza Roma,Lavalle y Bouchard,4,LAVALLE y BOUCHARD,12,5,-34.603205,-58.370342,0.701368,-147.892522,1.987223,True
4,-58.420997,-34.580127,Plaza Italia,Av. Santa Fe y Av. Sarmiento,5,SARMIENTO AV. y CALZADA CIRCULAR PLAZA ITALIA,42,11,-34.574162,-58.416990,0.807916,-514.242713,3.467381,True
5,-58.369709,-34.628301,Parque Lezama,Av Martin Garcia e Irala,6,"GARCIA, MARTIN AV. e IRALA",12,15,-34.627072,-58.373073,0.651503,15.859768,2.404949,False
6,-58.380990,-34.605840,Obelisco,Av. 9 de Julio y Tte. Gral Juan Domingo Peron,7,"PELLEGRINI, CARLOS y PERON, JUAN DOMINGO, TTE....",10,12,-34.602809,-58.382236,0.648369,-106.221429,1.669780,True
7,-58.389253,-34.609930,Congreso,Av. Hipolito Yrigoyen y Virrey Ceballos,8,"YRIGOYEN, HIPOLITO AV. y CEVALLOS, VIRREY",24,13,-34.610873,-58.391128,0.544670,62.702296,3.383327,False
8,-58.407145,-34.583915,Parque Las Heras,Parque Las Heras,9,"DIAZ, CNEL. AV. y FRENCH",24,23,-34.582460,-58.406626,0.668911,-325.316999,3.998101,True
9,-58.365746,-34.615551,Puerto Madero - UCA,Moreau de Justo 1500,10,"1500 MOREAU DE JUSTO, ALICIA AV.",12,14,-34.614912,-58.365744,0.670338,-41.851240,1.880165,True


In [10]:
average_fuller_clusters = stations_clusters_original[['cluster','lat_centroide','long_centroide','radius','bicicletas_en_estacion','usos']].copy()
#average_fuller_clusters["long_centroide"] = average_fuller_clusters["long_centroide"]
#print(average_fuller_clusters["long_centroide"].unique())
#print(average_fuller_clusters["long_centroide"].astype(str).unique())
#average_fuller_clusters = average_fuller_clusters[['long_centroide','bicicletas_en_estacion']]
average_fuller_clusters = average_fuller_clusters.groupby(['cluster','lat_centroide','long_centroide','radius'])['bicicletas_en_estacion','usos'].mean().reset_index()
average_fuller_clusters['very_used'] = np.where(average_fuller_clusters['bicicletas_en_estacion'] < 0, True, False)
average_fuller_clusters

,cluster,lat_centroide,long_centroide,radius,bicicletas_en_estacion,usos,very_used
0,0,-34.589985,-58.448490,0.660764,28.255604,1.643295,False
1,1,-34.588454,-58.393190,0.697636,13.833556,2.513785,False
2,2,-34.613722,-58.377679,0.759417,33.131849,2.155843,False
3,3,-34.628008,-58.424064,1.171162,33.470216,1.867048,False
4,4,-34.603248,-58.416929,0.709034,79.770969,2.379385,False
5,5,-34.603205,-58.370342,0.701368,29.558163,2.066305,False
6,6,-34.614012,-58.408327,1.049094,42.670739,2.201517,False
7,7,-34.634526,-58.361821,0.922874,19.883777,1.741665,False
8,8,-34.589007,-58.422305,0.627903,17.064780,2.428458,False
9,9,-34.635852,-58.408226,0.763370,7.309650,1.710615,False


In [53]:
# OLD WAY
# Adding a station in any place in the cluster radius
import random
cluster_info = """SELECT lat, long 
                FROM [pfidb].[dbo].[stations_clusters_usage]
                WHERE cluster=19"""

cluster_lat_long = engine.execute(cluster_info).fetchall()
lat_list = []
long_list = []
for i in engine.execute(cluster_info).fetchall():
    lat_list.append(i[0])
    long_list.append(i[1])
    
lat_aux = (random.choice(lat_list)+random.choice(lat_list))/2
while lat_aux in lat_list:
    lat_aux = (random.choice(lat_list)+random.choice(lat_list))/2
    
print(lat_aux)

-34.617004210000005


In [50]:
# NEW WAY
# Adding a station near the fuller station
### Adding new station to the cluster table and generating df_merged_radius from the DB
def jsonify_usig(res):
    return json.loads(res.text[1:-1])

def new_lat_long(lat1, lon1):
    # given: lat1, lon1, b = bearing in degrees, d = distance in kilometers
    b = random.randint(1, 360)
    d = 0.2
    origin = Point(lat1, lon1)
    destination = distance(kilometers=d).destination(origin, b)
    return destination


# ONLY NEEDS NAME, CAPACITY AND CLUSTER!!
new_station = {
    'long':0,
    'lat':0,
    'nombre':'Test 5 para variar cluster 15',
    'domicilio':'',
    'nro_est':0,
    'dire_norm':'',
    'capacidad':100,
    'cluster':18,
    'lat_centroide':0,
    'long_centroide':0
}

new_station['nro_est'] = engine.execute("SELECT MAX(nro_est)+1 FROM [estaciones-de-bicicletas-publicas]").fetchone()[0]

cluster_info_query = """SELECT TOP 1 lat, long, lat_centroide, long_centroide, radius
                        FROM [pfidb].[dbo].[stations_clusters_usage]
                        WHERE cluster={}
                        ORDER BY bicicletas_en_estacion ASC""".format(new_station['cluster'])

cluster_station_info = engine.execute(cluster_info_query).fetchone()
#new_station['lat'] = cluster_lat_long[0]
#new_station['long'] = cluster_lat_long[1]
centroid_location = (cluster_station_info[2],cluster_station_info[3])
new_station['lat_centroide'] = cluster_station_info[2]
new_station['long_centroide'] = cluster_station_info[3]



# Calculating new lat lon
destination = new_lat_long(cluster_station_info[0], cluster_station_info[1])
lat2, lon2 = destination.latitude, destination.longitude
new_station_location = (lat2, lon2)
print(new_station_location)
print(distance(new_station_location, centroid_location).km)

while distance(new_station_location, centroid_location).km >= cluster_station_info[4]:
    destination = new_lat_long(cluster_station_info[0], cluster_station_info[1])
    lat2, lon2 = destination.latitude, destination.longitude
    new_station_location = (lat2, lon2)
    

# lat_long_lists_query = """SELECT lat, long 
#                 FROM [pfidb].[dbo].[stations_clusters_usage]
#                 WHERE cluster={}""".format(new_station['cluster'])

# lat_list = []
# long_list = []
# for i in engine.execute(lat_long_lists_query).fetchall():
#     lat_list.append(i[0])
#     long_list.append(i[1])

# lat_aux = (random.choice(lat_list)+random.choice(lat_list))/2
# while lat_aux in lat_list:
#     lat_aux = (random.choice(lat_list)+random.choice(lat_list))/2


# long_aux = (random.choice(long_list)+random.choice(long_list))/2
# while long_aux in long_list:
#     long_aux = (random.choice(long_list)+random.choice(long_list))/2

new_station['lat'] = lat2
new_station['long'] = lon2

# Turning lat long to address
URL="https://ws.usig.buenosaires.gob.ar/geocoder/2.2/reversegeocoding"
# X is longitude and Y is latitude
params = {'x':new_station['long'], 'y':new_station['lat']}
usig_address = jsonify_usig(requests.get(URL,params=params))

new_station['domicilio'] = usig_address['puerta']
new_station['dire_norm'] = usig_address['puerta']


stations_clusters_query = """SELECT * FROM [stations_with_centroids]"""
df_merged_radius = pd.read_sql(stations_clusters_query,conn)
df_merged_radius = df_merged.append(new_station, ignore_index=True)
#insert_stations_with_centroids(engine,'stations_with_centroids',df_merged_radius)


aux_keys = ['long','lat','nombre','domicilio','nro_est','dire_norm','capacidad']
new_station_aux = { key: new_station[key] for key in aux_keys }
stations_query = """SELECT * FROM [estaciones-de-bicicletas-publicas]"""
df_stations = pd.read_sql(stations_query,conn)
df_stations = df_stations.append(new_station_aux, ignore_index=True)
#generic_insert(engine,'estaciones-de-bicicletas-publicas',df_stations)

#df_merged_radius = pd.read_sql(stations_clusters_query,conn)

print(new_station_aux)


(-34.59343834118049, -58.37318651809619)
0.3064928416055681
{'long': -58.37318651809619, 'lat': -34.59343834118049, 'nombre': 'Test 5 para variar cluster 15', 'domicilio': 'SAN MARTIN 1225', 'nro_est': 201, 'dire_norm': 'SAN MARTIN 1225', 'capacidad': 100}


In [55]:
lon=107648.050137
lat=104353.465453
URL="https://ws.usig.buenosaires.gob.ar/rest/convertir_coordenadas"
params = {'x':lon, 'y':lat,'output':'lonlat'}
corrected_lonlat = requests.get(URL,params=params).json()
corrected_lonlat = corrected_lonlat['resultado']
corrected_lonlat = (float(corrected_lonlat['y']),float(corrected_lonlat['x']))
print(corrected_lonlat)

(-34.589997, -58.379938)
